In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime

In [2]:
df=pd.read_csv('Combine.csv')
df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, '%d-%b-%y'))
# Convert win/loss outcomes to binary labels (1 = Team B, 0 = Team A)
df['Result'] = np.where(df['Result'] == 'Team B', 1, 0)
df['Time_48'] = df['Time_48'].astype(str)
df['Time_48']= df['Time_48'].str.replace('0 days', '')
df['Time_48']= df['Time_48'].str.replace('.000000000', '')
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return (h * 3600) + (m * 60) + s

df['Time_Sec']=df['Time_48'].apply(time_to_seconds)
### Creating Total Rebound
df['Team_A_RB']= df['Team_A_teamORB']+df['Team_A_teamDRB']+df['Team_A_ORB']+df['Team_A_DRB']
df['Team_B_RB']= df['Team_B_teamORB']+df['Team_B_teamDRB']+df['Team_B_ORB']+df['Team_B_DRB']

### NEW METRIC Points Attempted
df['Team_A_PTA']=(df['Team_A_FGA']-df['Team_A_3PA'])*2+df['Team_A_3PA']*3+df['Team_A_FTA']
df['Team_B_PTA']=(df['Team_B_FGA']-df['Team_B_3PA'])*2+df['Team_B_3PA']*3+df['Team_B_FTA']

### NEW METRIC 3P as % of all points
df['Team_A_3P%_All']=(df['Team_A_3P']*3)/(df['Team_A_Score'])
df['Team_B_3P%_All']=(df['Team_B_3P']*3)/(df['Team_B_Score'])
### Creating Difference of Stats (AST,BLK,TO,RBD)
df['diff_PT']=(df['Team_B_Score']-df['Team_A_Score'])
df['diff_AST']=(df['Team_B_AST']-df['Team_A_AST'])
df['diff_RB']=(df['Team_B_RB']-df['Team_A_RB'])
df['diff_ORB']=((df['Team_B_ORB'])+(df['Team_B_teamORB']))-((df['Team_A_ORB'])+(df['Team_A_teamORB']))
df['diff_DRB']=((df['Team_B_DRB'])+(df['Team_B_teamDRB']))-((df['Team_A_DRB'])+(df['Team_A_teamDRB']))
df['diff_ST']=(df['Team_B_ST']-df['Team_A_ST'])
df['diff_BLK']=(df['Team_B_BLK']-df['Team_A_BLK'])
df['diff_TO']=(df['Team_B_TO']-df['Team_A_TO'])
df['diff_PF']=(df['Team_B_PF']-df['Team_A_PF'])
df['diff_Tech']=(df['Team_B_Tech']-df['Team_A_Tech'])
df['diff_PTA']=(df['Team_B_PTA']-df['Team_A_PTA'])
df['diff_3P%_All']=df['Team_B_3P%_All']-df['Team_A_3P%_All']
### (0 = Away Team leading | 1= Home Team leading)
df['Home_Leading'] = np.where(df['Team_A_Score']<df['Team_B_Score'], 1, 0)

regular_season=df[df['Date']<='10-Apr-22']
playin_playoff=df[df['Date']>'10-Apr-22']

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Scale and normalize the input data, including the time variable
scaler = StandardScaler()
X = scaler.fit_transform(regular_season[['Time_Sec','diff_PT', 'diff_AST',
       'diff_RB', 'diff_ORB', 'diff_DRB', 'diff_ST', 'diff_BLK', 'diff_TO',
       'diff_PF', 'diff_Tech','diff_PTA','Home_Leading']])

# Define the target variable
y = regular_season['Result'].values

# Split the data into training and testing sets
train_size = int(0.8 * len(regular_season))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape the input data to fit the expected shape for an LSTM model
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Define and train the LSTM model
model = Sequential()
model.add(LSTM(32, input_shape=(1, X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

In [4]:
# save the model
model.save('LTSM_model.h5')

In [5]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print the test loss and accuracy
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)


Test loss: 0.5000759363174438
Test accuracy: 0.7787521481513977
